Original code from https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py under the following license:

MIT License

Copyright (c) 2017 Erik Linder-Norén

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


The author's model was based on the paper here: https://arxiv.org/pdf/1511.06434.pdf

In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Global Constants
images_dir = "dcgan_images"
img_rows = 800
img_cols = 640
channels = 1
noise_len = 100

## Define functions for creating, training, and using the models

In [4]:
def build_discriminator():
    '''
    Put together a CNN that will return a single confidence output.
    
    returns: the model object
    '''

    img_shape = (img_rows, img_cols, channels)

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [5]:
def build_generator():
    '''
    Put together a model that takes in one-dimensional noise and outputs two-dimensional data representing a black
    and white image, with -1 for black and 1 for white.
    
    returns: the model object
    '''

    noise_shape = (noise_len,)

    model = Sequential()

    model.add(Dense(128 * int(img_rows/4) * int(img_cols/4), activation="relu", input_shape=noise_shape))
    model.add(Reshape((int(img_rows/4), int(img_cols/4), 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8)) 
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(1, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    return model

In [6]:
def build_combined():
    '''
    Puts together a model that combines the discriminator and generator models.
    
    returns: the generator, discriminator, and combined model objects
    '''
    
    optimizer = Adam(0.0002, 0.5)

    # Build and compile the discriminator
    discriminator = build_discriminator()
    discriminator.compile(loss='binary_crossentropy', 
                          optimizer=optimizer,
                          metrics=['accuracy'])


    # Build and compile the generator
    generator = build_generator()
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    # The generator takes noise as input and generates images
    noise = Input(shape=(noise_len,))
    img = generator(noise)
    
    # For the combined model we will only train the generator
    discriminator.trainable = False

    # The discriminator takes generated images as input and determines validity
    valid = discriminator(img)

    # The combined model  (stacked generator and discriminator) takes
    # noise as input => generates images => determines validity 
    combined = Model(inputs=noise, outputs=valid)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator, discriminator, combined

In [7]:
def save_imgs(generator, epoch):
    '''
    Has the generator create images and saves the images in a single file that includes the epoch in the filename.
    
    inputs:
        generator: the generator model object returned by build_combined
        epoch: the epoch number (but can be anything that can be represented as a string)
    
    returns: None
    '''
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, noise_len))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    #fig.suptitle("DCGAN: Generated digits", fontsize=12)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(os.path.join(images_dir, 'mnist_{}.png'.format(epoch)))
    plt.close()

In [8]:
def train(generator, discriminator, combined, epochs, batch_size=128, save_interval=50):
    '''
    Trains all model objects
    
    generator: the generator model object returned by build_combined
    discriminator: the discriminator model object returned by build_combined
    combined: the combined model object returned by build_combined
    epochs: integer, the number of epochs to train for
    batch_size: integer, the number of training samples to use at a time
    save_interval: integer, will generate and save images when the current epoch % save_interval is 0
    
    returns: None
    '''
    
    dataset = []
    for i in range(1000):
        image = []
        for j in range(img_rows):
            number = np.random.random()*2-1
            row = []
            for k in range(img_cols):
                row.append([number])
            image.append(row)
#             image.append(row)
#             image.append(row)
#             image.append(row)
        dataset.append(image)
    new = []
    for i in range(1000):
        number = np.random.randint(0,3)
        n1 = np.random.random()*2-1
        image = []
        if number == 0:
            image = [[[0],[0],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[n1],[0],[0],[0]],
                     [[0],[n1],[0],[0],[n1],[0],[0],[0],[0]],
                     [[0],[n1],[0],[n1],[0],[0],[0],[0],[0]],
                     [[0],[n1],[n1],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[n1],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[n1],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[n1],[0],[0],[0]],
                     [[0],[0],[0],[0],[0],[0],[0],[0],[0]]]
        elif number == 1:
            image = [[[0],[0],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[n1],[0],[0],[0]],
                     [[0],[0],[n1],[0],[n1],[0],[0],[0],[0]],
                     [[0],[0],[0],[n1],[0],[0],[0],[0],[0]],
                     [[0],[0],[0],[n1],[0],[0],[0],[0],[0]],
                     [[0],[0],[0],[n1],[0],[0],[0],[0],[0]],
                     [[0],[0],[0],[n1],[0],[0],[0],[0],[0]],
                     [[0],[0],[0],[n1],[0],[0],[0],[0],[0]],
                     [[0],[0],[0],[0],[0],[0],[0],[0],[0]]]
        else:
            image = [[[0],[0],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[0],[0],[0],[0],[0],[0],[0]],
                     [[0],[n1],[n1],[n1],[n1],[n1],[0],[0],[0]],
                     [[0],[0],[0],[0],[0],[0],[0],[0],[0]]]
        new.append(image)
        
        
        img = Image.open("josh.jpg")
        width, height = img.size
        pixels = img.getdata()
        joshs = []
        for i in range(1000):
            josh = []
            for y in range(height):
                row = []
                for x in range(width):
                    r, g, b = pixels[idx]
#                     int rgb = red;
#                     rgb = (rgb << 8) + green;
#                     rgb = (rgb << 8) + blue;
                    row.append([r/255*2-1])
                josh.append(row)
            joshs.append(josh)
        
        
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

#     Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
#     print(X_train[0][0],"train",dataset[0][0])
#     X_train=dataset
    X_train = joshs
    half_batch = int(batch_size / 2)

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of images
#         idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[:half_batch]

        # Sample noise and generate a half batch of new images
        noise = np.random.normal(0, 1, (half_batch, noise_len))
        gen_imgs = generator.predict(noise)

        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

        # ---------------------
        #  Train Generator
        # ---------------------

        noise = np.random.normal(0, 1, (batch_size, noise_len))

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

        # If at save interval => save generated image samples and plot progress
        if epoch % save_interval == 0:
            # Plot the progress
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            print ("{} [D loss: {}, acc.: {:.2%}] [G loss: {}]".format(epoch, d_loss[0], d_loss[1], g_loss))
            save_imgs(generator, epoch)

In [9]:
def show_new_image(generator):
    '''
    Generates and displays a new image
    
    inputs: generator object model returned from build_combined
    
    returns: generated image
    '''
    
    noise = np.random.normal(0, 1, (1, noise_len))
    gen_img = generator.predict(noise)[0][:,:,0]
    
    return plt.imshow(gen_img, cmap='gray', vmin=-1, vmax=1)

## This is the main section of the code, that actually creates the model objects

In [10]:
# set up directories to hold the images that are saved during training checkpoints.
import os

if (not os.path.isdir(images_dir)):
    os.mkdir(images_dir)

In [11]:
# Uncomment if you want to build your own new models
generator, discriminator, combined = build_combined()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


**Make sure that you have downloaded the three h5 files before running the next block.**

In [ ]:
# Load the saved model files. Comment out (or don't run) this block if you want to start with fresh models.
# from keras.models import load_model

# generator = load_model('generator.h5')
# discriminator = load_model('discriminator.h5')
# combined = load_model('combined.h5')

In [ ]:
train(generator, discriminator, combined, epochs=201, batch_size=32, save_interval=50)

In [ ]:
show_new_image(generator)

In [ ]:
generator.save('generator.h5')
discriminator.save('discriminator.h5')
combined.save('combined.h5')